In [ ]:
import yaml
import rdflib
import pandas as pd

In [ ]:
with open("map.yml", "r") as f:
    data = yaml.safe_load(f.read())

In [ ]:
g = rdflib.Graph()
ns_lookup = {}

for ns in data['namespaces']:
    g.namespace_manager.bind(namespace=ns['uri'], prefix=ns['name'])
    ns_lookup[ns['name']] = rdflib.Namespace(ns['uri'])


In [ ]:
def csv_to_graph(mappings, filename, graph):
    if mappings:
        for row in pd.read_csv(filename).to_dict(orient="records"):
            uriref = mappings.get('uriref')
            if uriref:
                identifier = rdflib.URIRef(row.get(uriref))
            else:
                identifier = rdflib.BNode()
            for key, value in ((key, value) for key, value in mappings.items() if 'uriref' not in key):
                ns, ns_attr = key.split(".")
                if type(value) is list:
                    for val in value:
                        if not pd.isna(row[val]):
                            graph.add( (identifier, ns_lookup[ns][ns_attr], rdflib.Literal(row[val])) )
                elif type(value) is dict:
                    sub_uriref = row[value.get('uriref')]
                    if not pd.isna(sub_uriref):
                        graph.add( (identifier, ns_lookup[ns][ns_attr], rdflib.URIRef(sub_uriref)) )
                else:
                    if not pd.isna(row[value]):
                        graph.add( (identifier, ns_lookup[ns][ns_attr], rdflib.Literal(row[value])) )

for data_file in data['mappings']:
    filename = f"data/{data_file}.csv"
    mappings = data['mappings'][data_file]
    csv_to_graph(mappings, filename, g)


In [ ]:
g.serialize(destination='output/triple.txt', format='turtle')

In [ ]:
qres = g.query(
    """SELECT *
       WHERE {
          ?s ?p ?o .
          FILTER(regex(?o, "Hilarius Arelatensis", "i"))
       }""")

In [ ]:
for row in qres:
    print(row)

In [ ]:
from flask import Flask, request
from json import dumps
app = Flask(__name__)

@app.route('/query', methods=['POST'])
def hello_world():
    sparql_query = request.get_data()
    query_results = g.query(sparql_query)
    return dumps(list(query_results))

if __name__ == '__main__':
    app.run()